In [1]:
import requests
import json
import random
import numpy as np
import pandas as pd
import math
import time
import serial

from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from collections import Counter
from numpy import arctan2,random,sin,cos,degrees

url = 'http://odapapi.defrilab.com/api/Odap/recentMessages'
count=1

while(True):
    print('Going for process, round {}'.format(count))
    response = requests.get(url)
    data = response.json()
    print('Printing the first 5 elements\n')
    print(data[:5])
    loc_compact  = []
    for loc in data:
        loc_compact.append([loc['lat'],loc['long']])
    loc_compact = np.array(loc_compact)
    print('Ploting Scatter Plot\n')
    plt.scatter(loc_compact[:,0], loc_compact[:,1])
    wcss = []
    print('Determining the optimul number of clusters using the elbow method\n')
    for i in range(1, 11):
        kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=len(loc_compact), n_init=10, random_state=0)
        kmeans.fit(loc_compact)
        wcss.append(kmeans.inertia_)
    plt.plot(range(1, 11), wcss)
    plt.title('Elbow Method')
    plt.xlabel('Number of clusters')
    plt.ylabel('WCSS')
    plt.show()  
    print('Determining the centroid for each clustern\n')
    kmeans = KMeans(n_clusters=2, init='k-means++', max_iter=len(loc_compact), n_init=10, random_state=0)
    pred_y = kmeans.fit_predict(loc_compact)
    plt.scatter(loc_compact[:,0], loc_compact[:,1])
    plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s=len(loc_compact), c='red')
    plt.show()
    centroids = kmeans.cluster_centers_
    labels =list(kmeans.labels_)
    classes = list(set(labels))
    unique = Counter(labels)
    bigger_cluster = max(unique, key=unique.get)
    centroids = centroids[bigger_cluster]
    lat = centroids[0]
    long = centroids[1]
    print('Calculating the bearing of the centroid of the biggest cluster\n')
    a = {'lat': -0.399754, 'long': 36.96256867}
    b = {'lat': lat, 'long': long}
    dL = b['long'] - a['long']
    X = cos(b['lat'])* sin(dL)
    Y = cos(a['lat'])*sin(b['lat']) - sin(a['lat'])*cos(b['lat'])* cos(dL)
    bearing_rad = arctan2(X,Y)
    bearing_degrees = bearing_rad * (180/math.pi)
    print('Sending the coordinates of the Biggest Cluster Centroid to Arduino\n')
    arduino = serial.Serial(port='/dev/ttyACMO', baudrate=115200, timeout=.1)
    arduino.write(bytes(bearing_degrees, 'utf-8'))    
    print('Data sent to Arduino, waiting for response\n')
    time.sleep(0.05)
    response = arduino.readline()
    if(response=='ok'):
        print('Arduino just responded with Ok, data transmission was successful!\n')
        print('Process round {} completed successfully.'.format(count))
    else:
        print('Arduino is not responding for now, we wil try re-establish connection the next round, hope She will respond then\n')
    count +=1
    print('Going into sleep for 6 seconds before redoing the whole process\n')
    time.sleep(6)

Going for process, round 1


ConnectionError: HTTPConnectionPool(host='odapapi.defrilab.com', port=80): Max retries exceeded with url: /api/Odap/recentMessages (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f72197a90>: Failed to establish a new connection: [Errno -2] Name or service not known'))